<a href="https://colab.research.google.com/github/Le00liveira02/Web-scraping-KNN/blob/main/Zappos_web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


#Web Scraper

In [ ]:
resultados=[]
pgs=50
categorias=['women-boots-shoes','women-sneakers-athletic-shoes','women-sandals-shoes','women-heels-shoes','women-flats-shoes','women-comfort-shoes','men-boots-shoes','men-sneakers-athletic-shoes','men-sandals-shoes','men-sneakers-athletic-shoes','men-oxfords-shoes']



for cat in categorias:

   for pg in range(pgs):
      pagina='https://www.zappos.com/'+ cat +'/.zso?t='+cat+'&p='+ str(pg)
      response=requests.get(pagina)
      site=BeautifulSoup(response.text,'html.parser')
      produtos=site.findAll('article',attrs={'class':'tD-z'})
              

      for produto in produtos:

          marca=produto.find('dd',attrs={'class':'WC-z'})
          modelo=produto.find('dd',attrs={'class':'XC-z'})
          cor=produto.find('dd',attrs={'class':'YC-z'})
          preco=produto.find('span',attrs={'class':'UD-z'})
          estrelas=produto.find('span',attrs={'class':'yl-z'})
          votos=produto.find('span',attrs={'class':'zl-z'})
          #imagem=produto.find('img',attrs={'class':'kS-z Ri-z'}) #imagem['src']
          #imagem2=produto.find('img',attrs={'class':'kS-z Ri-z Si-z'})

  

          if (estrelas):
            resultados.append([marca.text, modelo.text, cor.text, preco['content'], estrelas.text, votos.text, cat, pagina])
          else:
            resultados.append([marca.text, modelo.text, cor.text, preco['content'], "Não Classificado", 'Sem votos', cat, pagina])





Montando o Data Frame

In [ ]:
df=pd.DataFrame(resultados,columns=['Marca','Modelo','cor','Preco','Estrelas','Votos', 'Categoria','Link da pagina'])

Limpando os dados

In [ ]:
df['Votos'] = df['Votos'].apply(lambda x:x.replace("(",""))
df['Votos'] = df['Votos'].apply(lambda x:x.replace(")",""))
df['Excluir']=df['Modelo'].apply(lambda x:"Jibbitz" in x)#Jibbitz são enfeites para marca Crocs
df=df.drop(df[df['Excluir']==True].index)
df=df.drop(columns='Excluir')

#Knn

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

#Calculo da média de preço por categoria
med_preco=df.groupby(['Marca','Categoria']).agg(preco_mean=('Preco','mean')).reset_index()
med_preco=pd.pivot_table(med_preco, values='preco_mean', index='Marca',columns=['Categoria'],fill_value=0)

#Calculo da média de estrelas por categoria
med_est=df.drop(df[df['Estrelas']=='Não Classificado'].index)
med_est['Estrelas']=med_est['Estrelas'].astype('float')
med_est=med_est.groupby(['Marca','Categoria']).agg({'Estrelas':['mean']}).reset_index()
med_est = pd.pivot_table(med_est, values='Estrelas', index='Marca',columns=['Categoria'],fill_value=0)

#Unificando os calculos
df=df[['Marca']].drop_duplicates()
df=pd.merge(df,med_preco,on='Marca',how='left')
df=pd.merge(df,med_est,on='Marca',how='left')

df=df.fillna(0)

In [ ]:
#Guardando index e setando a marca como index
f_knn_index=df['Marca']
df=df.set_index('Marca')

#Normalizando os dados devido a diferença de escala ente votos e preço
normalizar=MinMaxScaler(feature_range=(0,1))
df_knn= normalizar.fit_transform(df)  
            
df_knn=pd.DataFrame(df_knn,index=df_knn_index)

In [ ]:
#KNN
modelo_knn = NearestNeighbors(metric='cosine')
modelo_knn.fit(df_knn)

In [ ]:
#Estrutura para gerar csv com as recomendações
df_net=pd.DataFrame()
marcas=df_knn_index.tolist()

for marca in marcas:
  distancia,vizinhos=modelo_knn.kneighbors(df_knn.loc[[marca]],n_neighbors=6)
  dist=distancia.tolist()[0]
  vizinhos=vizinhos.tolist()[0]
  vizinhos=df_knn.iloc[vizinhos[0:6]].index.to_list()

  b={'Marca':marca,'vizinho':vizinhos,'dist':dist}
  s=pd.DataFrame.from_dict(b)

  df_net=df_net.append(s, ignore_index=True)

In [ ]:
df_net.to_csv('net.csv',sep=',')